Install packages:

In [ ]:
!pip install --upgrade sacremoses
!pip install transformers[torch]
!pip install accelerate -U
!pip install --upgrade seqeval
!pip install --upgrade datasets
!pip install --upgrade evaluate
!pip install --upgrade tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00


---------------------
load the serialized corpus ready for experimenting, convert to huggingface:

In [ ]:
import pickle
import datasets

dataset = pickle.load(open('hf-dataset_v2.pkl','rb'))
dataset_hf = datasets.Dataset.from_list(dataset)

unique_tags = set([])
[ unique_tags.update(item['labels']) for item in dataset ]
print(unique_tags)
nr_classes = len(unique_tags)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
label_names = list(tag2id.keys())

{'aggression', 'O'}


In [ ]:
dataset_hf

Dataset({
    features: ['tokens', 'labels', 'fname'],
    num_rows: 802
})

auxilliary functions to compute metrics and tokenization

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EvalPrediction

import numpy as np

from evaluate import load
metric = load("seqeval")

def compute_metrics(p: EvalPrediction):
    predictions = p.predictions
    labels = p.label_ids
    #predictions = np.argmax(predictions, axis=2)
    predictions = predictions.argmax(-1)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results


#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split, tokenizer):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True, max_length=512) #, padding='max_length', pad_to_max_length=True, max_length=512)

  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["labels"][k]
    existing_label_ids = [tag2id[label] for label in existing_label_ids] # line added as we need numeric label ids
    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## HerBERT-large
The cell below contains 10-fold cross validation experiments using HerBERT-large:

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import KFold, train_test_split
from transformers import EvalPrediction, BertConfig, AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from scipy.stats import t
import datasets, torch, gc, transformers, os
import wandb


transformers.logging.set_verbosity_error()
os.environ["WANDB_DISABLED"] = "true"


def compute_metrics_sklearn(p: EvalPrediction):
    predictions = p.predictions.argmax(-1)
    labels = p.label_ids

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_preds = [item for sublist in true_predictions for item in sublist]
    flat_labels = [item for sublist in true_labels for item in sublist]

    f1 = f1_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    recall = recall_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    precision = precision_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    return {"f1": f1, "recall": recall, "precision": precision}


training_args = TrainingArguments(
    do_eval=True,
    output_dir='./aggression_model',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=20,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=80,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=80,
    eval_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
    save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
    # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
    learning_rate=5e-5,
    eval_steps = 20,
    save_total_limit = 1,
    load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
    metric_for_best_model='eval_f1',
    disable_tqdm=False,
    report_to=None
)


n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

f1_scores = []
recall_scores = []
precision_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(dataset_hf)):

    model_config = BertConfig.from_pretrained("allegro/herbert-large-cased", num_labels=len(id2tag), id2label=id2tag,  label2id=tag2id)
    tokenizer_obj = AutoTokenizer.from_pretrained("allegro/herbert-large-cased", do_lower_case=False)
    model = BertForTokenClassification.from_pretrained("allegro/herbert-large-cased", config=model_config)


    print(f"Fold {fold+1}/{n_splits}")

    train_val_dataset = dataset_hf.select(train_index)
    test_dataset = dataset_hf.select(test_index)

    train_dataset, val_dataset = train_val_dataset.train_test_split(test_size=0.125, shuffle=True).values()

    tokenized_train_dataset = train_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)
    tokenized_val_dataset = val_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)
    tokenized_test_dataset = test_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)

    data_collator = DataCollatorForTokenClassification(tokenizer_obj, return_tensors='pt') #, padding='max_length')

    # Retrain model on current fold
    trainer = Trainer(
        model=model,  # Use the model architecture from before
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer_obj,
        compute_metrics=compute_metrics_sklearn # use sklearn metrics
    )
    trainer.train()

    # Evaluate on test set
    eval_results = trainer.predict(tokenized_test_dataset)
    metrics = compute_metrics_sklearn(eval_results)

    f1_scores.append(metrics["f1"])
    recall_scores.append(metrics["recall"])
    precision_scores.append(metrics["precision"])

    del model
    torch.cuda.empty_cache()
    gc.collect()

# Calculate mean and confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), np.std(a, ddof=1)/np.sqrt(n)
    h = se * t.ppf((1 + confidence)/2., n-1)
    return m, m-h, m+h

f1_mean, f1_lower, f1_upper = mean_confidence_interval(f1_scores)
recall_mean, recall_lower, recall_upper = mean_confidence_interval(recall_scores)
precision_mean, precision_lower, precision_upper = mean_confidence_interval(precision_scores)

print(f"F1: Mean={f1_mean:.4f}, 95% CI=({f1_lower:.4f}, {f1_upper:.4f})")
print(f"Recall: Mean={recall_mean:.4f}, 95% CI=({recall_lower:.4f}, {recall_upper:.4f})")
print(f"Precision: Mean={precision_mean:.4f}, 95% CI=({precision_lower:.4f}, {precision_upper:.4f})")


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

...

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

<ipython-input-15-86b4ef44dbab>:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,F1,Recall,Precision
20,No log,0.353932,0.838000,0.889860,0.791851
40,No log,0.271608,0.886761,0.889257,0.884595
60,No log,0.202353,0.910854,0.915511,0.908451
80,0.365500,0.202087,0.907971,0.919104,0.909802
100,0.365500,0.308822,0.914781,0.921867,0.913763
120,0.365500,0.248317,0.915550,0.922545,0.914610
140,0.365500,0.316006,0.906456,0.911441,0.903741
160,0.107600,0.315049,0.914830,0.921164,0.913298
180,0.107600,0.327622,0.916408,0.916064,0.916767
200,0.107600,0.330978,0.915315,0.919581,0.913211


F1: Mean=0.8967, 95% CI=(0.8857, 0.9077)
Recall: Mean=0.9028, 95% CI=(0.8935, 0.9121)
Precision: Mean=0.8952, 95% CI=(0.8842, 0.9062)


## Bi-LSTM
The cells below are required for 10-fold cross validation experiments using the Bi-LSTM approach:

In [ ]:
# prompt: given the same c-v split of
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
# and dataset_hf, the same splits to  train_val_dataset and test_dataset, propose a code to train and evaluate a Bi-LSTM model linear model. Use torch and transformers packages. Evaluate models using the same metrics as in the previous cell, use mean_confidence_interval.

from sklearn.metrics import f1_score, recall_score, precision_score
from transformers import EvalPrediction, BertConfig, AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

import numpy as np
from scipy.stats import t

def compute_metrics_sklearn(p: EvalPrediction):
    predictions = p.predictions.argmax(-1)
    labels = p.label_ids

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_preds = [item for sublist in true_predictions for item in sublist]
    flat_labels = [item for sublist in true_labels for item in sublist]

    f1 = f1_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    recall = recall_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    precision = precision_score(flat_labels, flat_preds, average='weighted', zero_division=0)
    return {"f1": f1, "recall": recall, "precision": precision}



In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import DataCollatorForTokenClassification
from transformers import EvalPrediction, BertConfig, AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

from sklearn.model_selection import KFold, train_test_split
import torch, os
import torch.nn as nn
import numpy as np

# Calculate mean and confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), np.std(a, ddof=1) / np.sqrt(n)
    h = se * t.ppf((1 + confidence) / 2., n - 1)
    return m, m - h, m + h

def count_total_parameters(model):
    return sum(p.numel() for p in model.parameters())

# 1. Define custom Config
class BiLSTMTokenClassificationConfig(PretrainedConfig):
    model_type = "bilstm_token_classification"

    def __init__(
        self,
        vocab_size=30522,
        hidden_size=256,
        num_labels=9,
        num_lstm_layers=1,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.num_lstm_layers = num_lstm_layers
        self.dropout = dropout

# 2. Define BiLSTM Model
class BiLSTMTokenClassifier(PreTrainedModel):
    config_class = BiLSTMTokenClassificationConfig

    def __init__(self, config):
        super().__init__(config)
        self.embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.lstm = nn.LSTM(
            input_size=config.hidden_size,
            hidden_size=config.hidden_size // 2,
            num_layers=config.num_lstm_layers,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        embeds = self.embeddings(input_ids)  # (batch_size, seq_len, hidden_size)
        lstm_out, _ = self.lstm(embeds)       # (batch_size, seq_len, hidden_size)
        lstm_out = self.dropout(lstm_out)
        logits = self.classifier(lstm_out)    # (batch_size, seq_len, num_labels)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Flatten logits and labels for loss computation
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.config.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return {"loss": loss, "logits": logits}

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

f1_scores = []
recall_scores = []
precision_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(dataset_hf)):

    print(f"Fold {fold+1}/{n_splits}")

    train_val_dataset = dataset_hf.select(train_index)
    test_dataset = dataset_hf.select(test_index)

    train_dataset, val_dataset = train_val_dataset.train_test_split(test_size=0.125, shuffle=True).values()

    tokenizer_obj = AutoTokenizer.from_pretrained("allegro/herbert-large-cased", do_lower_case=False)

    tokenized_train_dataset = train_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)
    tokenized_val_dataset = val_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)
    tokenized_test_dataset = test_dataset.map(lambda p: tokenize_adjust_labels(p, tokenizer_obj), batched=True)

    data_collator = DataCollatorForTokenClassification(tokenizer_obj, return_tensors='pt') #, padding='max_length')

    # Initialize and train the Bi-LSTM model
    config = BiLSTMTokenClassificationConfig(
        vocab_size=len(tokenizer_obj),
        hidden_size=256,
        num_labels=nr_classes,
        num_lstm_layers=len(id2tag),
        dropout=0.1,
    )

    model = BiLSTMTokenClassifier(config)
    model.to(device) # Assuming you have a device defined
    total_params = count_total_parameters(model)
    print(f"Total parameters: {total_params:,}")

    training_args = TrainingArguments(
      output_dir="./bilstm-token-classifier",
      overwrite_output_dir=True,
      num_train_epochs=10,              # total number of training epochs
      per_device_train_batch_size=20,  # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      warmup_steps=80,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=80,
      eval_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
      save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
      # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
      learning_rate=5e-5,
      eval_steps = 20,
      save_total_limit = 1,
      load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      metric_for_best_model='eval_f1',
      disable_tqdm=False,
      report_to=None
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer_obj,
        compute_metrics=compute_metrics_sklearn
    )
    trainer.train()

    # Evaluate on the test set
    eval_results = trainer.predict(tokenized_test_dataset)
    metrics = compute_metrics_sklearn(eval_results)

    f1_scores.append(metrics["f1"])
    recall_scores.append(metrics["recall"])
    precision_scores.append(metrics["precision"])


# Calculate and print the metrics with confidence intervals
f1_mean, f1_lower, f1_upper = mean_confidence_interval(f1_scores)
recall_mean, recall_lower, recall_upper = mean_confidence_interval(recall_scores)
precision_mean, precision_lower, precision_upper = mean_confidence_interval(precision_scores)

print(f"F1: Mean={f1_mean:.4f}, 95% CI=({f1_lower:.4f}, {f1_upper:.4f})")
print(f"Recall: Mean={recall_mean:.4f}, 95% CI=({recall_lower:.4f}, {recall_upper:.4f})")
print(f"Precision: Mean={precision_mean:.4f}, 95% CI=({precision_lower:.4f}, {precision_upper:.4f})")


Using device: cuda
Fold 1/10


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Total parameters: 13,591,042


<ipython-input-9-906f323b663e>:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,F1,Recall,Precision
20,No log,0.710652,0.241064,0.222365,0.808107
40,No log,0.686279,0.687692,0.618104,0.802602
60,No log,0.643751,0.830537,0.868606,0.800232
80,0.675700,0.571779,0.837533,0.888730,0.796271
100,0.675700,0.465976,0.837987,0.889835,0.791849
120,0.675700,0.387149,0.838000,0.889860,0.791851
140,0.675700,0.369591,0.838000,0.889860,0.791851
160,0.453000,0.365523,0.838000,0.889860,0.791851
180,0.453000,0.361778,0.838000,0.889860,0.791851
200,0.453000,0.358345,0.838000,0.889860,0.791851


F1: Mean=0.8068, 95% CI=(0.7870, 0.8266)
Recall: Mean=0.8680, 95% CI=(0.8542, 0.8819)
Precision: Mean=0.7538, 95% CI=(0.7297, 0.7779)
